In [1]:
import os

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline
import keras
import mne
import numpy as np
import pandas as pd
from scipy.io import loadmat
import tensorflow as tf
import random

from mne.channels import make_standard_montage
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model


In [2]:
data_dir = os.path.dirname("./data/")
data_files = os.listdir(data_dir)

In [3]:
def annotations_from_eGUI(raw, egui):
    codes = []
    starts = []

    current_state = None

    for i in range(len(egui)):
        if egui[i][0] != current_state:
            starts.append(i)
            current_state = egui[i][0]
            codes.append(str(egui[i][0]))

    starts.append(len(egui))
    codes = np.array(codes)
    sf = raw.info.get('sfreq')
    starts = np.array(starts) / sf
    durations = starts[1:] - starts[:-1]
    starts = starts[:-1]

    raw.set_annotations(mne.Annotations(onset=starts, duration=durations, description=codes))


def raw_from_mat(file):
    mat = loadmat(os.path.join(data_dir, file))

    sampling_freq = mat["o"][0][0][2][0][0]
    n_samples = mat["o"][0][0][3][0][0]
    ch_names = [element[0][0] for element in mat["o"][0][0][6]]

    df = pd.DataFrame(mat["o"][0][0][5], columns=ch_names)
    df = df.drop(columns=["X5"])
    df = df.T
    ch_names.remove("X5")

    ch_types = ['eeg'] * 21
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    raw = mne.io.RawArray(df.to_numpy(), info)

    montage = make_standard_montage("standard_prefixed")
    raw.set_montage(montage)

    raw.load_data().set_eeg_reference(ref_channels='average')
    annotations_from_eGUI(raw, mat["o"][0][0][4])
    return raw


def filter_raw(raw):
    return raw.load_data().filter(0.1, 30, method="fir", phase="zero-double")

In [4]:
raw_NoMT = [raw_from_mat(file) for file in data_files if "NoMT" in file]
raw_FREEFORM = [raw_from_mat(file) for file in data_files if "FREEFORM" in file]

Creating RawArray with float64 data, n_channels=21, n_times=664400
    Range : 0 ... 664399 =      0.000 ...  3321.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=664600
    Range : 0 ... 664599 =      0.000 ...  3322.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=662400
    Range : 0 ... 662399 =      0.000 ...  3311.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=667600
    Range : 0 ... 667599 =      0.000 ...  3337.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 d

In [5]:
def get_epochs(raw, event_id):
    metadata_tmin, metadata_tmax = -1, 1
    all_events, all_event_id = mne.events_from_annotations(raw, event_id=event_id)
    metadata, events, event_id = mne.epochs.make_metadata(
        events=all_events,
        event_id=event_id,
        tmin=metadata_tmin,
        tmax=metadata_tmax,
        sfreq=raw.info["sfreq"],
    )
    print(raw.info["sfreq"])
    return mne.Epochs(raw, events, event_id)


In [6]:
epochs_NoMT = [get_epochs(file, {"0": 1}) for file in raw_NoMT]
epochs_FREEFORM = [get_epochs(file, {'1': 2, '2': 3}) for file in raw_FREEFORM]
epochs_NOMT_only_code_2 = [get_epochs(file, {"2": 3}) for file in raw_NoMT]

Used Annotations descriptions: ['0']
200.0
Not setting metadata
966 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0']
200.0
Not setting metadata
960 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0']
200.0
Not setting metadata
963 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0']
200.0
Not setting metadata
968 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0']
200.0
Not setting metadata
968 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)

In [7]:
epochs_NoMT[0].get_data().max()

Using data from preloaded Raw for 966 events and 141 original time points ...
1 bad epochs dropped


865.3172938443672

In [8]:
epochs_FREEFORM[0].get_data().max()

Using data from preloaded Raw for 739 events and 141 original time points ...
0 bad epochs dropped


83.5072706155633

In [9]:
epochs_data_NOMT = [file.get_data() for file in epochs_NoMT]
epochs_data_FREEFORM = [file.get_data() for file in epochs_FREEFORM]
epochs_Data_NOMT_2 = [file.get_data() for file in epochs_NOMT_only_code_2]

Using data from preloaded Raw for 965 events and 141 original time points ...
Using data from preloaded Raw for 960 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 963 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 968 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 968 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 968 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 967 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 739 events and 141 original time points ...
Using data from preloaded Raw for 688 events and 141 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 700 events and 141 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 159 events and 141

In [10]:
stacked_NOMT = np.vstack(epochs_data_NOMT)
stacked_FREEFORM = np.vstack(epochs_data_FREEFORM)
stacked_NOMT_2 = np.vstack(epochs_Data_NOMT_2)

In [11]:
print(stacked_NOMT.shape)
print(stacked_FREEFORM.shape)
print(stacked_NOMT_2.shape)

(6753, 21, 141)
(2127, 21, 141)
(1114, 21, 141)


In [12]:
# import matplotlib.pyplot as plt
#
# plt.hist(stacked_NOMT.reshape(-1), bins=np.arange(-25, 25), density=True)
# plt.hist(stacked_FREEFORM.reshape(-1), bins=np.arange(-25, 25), density=True)
# #plt.hist(stacked_NOMT_2.reshape(-1), bins=np.arange(-25,25),density=True)
# plt.show()

In [13]:
np.random.shuffle(stacked_NOMT)
np.random.shuffle(stacked_FREEFORM)
np.random.shuffle(stacked_NOMT_2)


In [14]:
X_nomt_train = stacked_NOMT[:5000]
X_nomt_test = stacked_NOMT[5000:]

In [15]:
X_free = stacked_FREEFORM

In [16]:
X_nomt_2 = stacked_NOMT_2


In [17]:
# make Freeform test set same length as NoMT
idy = random.sample(range(0, len(X_free)), X_nomt_test.shape[0])
X_free_test = X_free[idy]

In [18]:
print(X_nomt_train.shape)
print(X_nomt_test.shape)
print(X_free_test.shape)
print(X_nomt_2.shape)

(5000, 21, 141)
(1753, 21, 141)
(1753, 21, 141)
(1114, 21, 141)


# Helper functions

In [19]:
def calc_accuracy(a, b, th):
    first = [1 if i < th else 0 for i in a]
    last = [1 if i > th else 0 for i in b]
    return sum(first + last) / len(first + last)

In [20]:
def calc_reconstruction_error(ae, A, B):
    err = []
    err2 = []
    for i in A:
        # need to expand here because the flatten layer assumes that the first dimension is the number of samples
        i = np.expand_dims(i, axis=0)
        err.append((np.square(i - ae.call(i))).mean())
    print("###################")

    for j in B:
        j = np.expand_dims(j, axis=0)
        err2.append((np.square(j - ae.call(j))).mean())
    print("##############")
    print(np.array(err).mean())
    print(np.array(err2).mean())
    return err, err2

# Standard Autoencoder

In [21]:
norm_layer_nomt = layers.Normalization()
norm_layer_free = layers.Normalization()
norm_layer_nomt_2 = layers.Normalization()

norm_layer_nomt.adapt(X_nomt_train.astype(float))
norm_layer_free.adapt(X_free_test.astype(float))
norm_layer_nomt_2.adapt(X_nomt_2.astype(float))

print(X_nomt_train)
print(np.max(X_nomt_train))
print(np.max(X_free_test))
print(np.max(norm_layer_nomt(X_nomt_train)))
print(np.max(norm_layer_free(X_free_test)))

[[[ -1.05243902  -1.00339141   0.74470383 ...  -6.89053426  -5.99624855
    -5.30005807]
  [ -4.25609756  -5.85704994  -3.9389547  ... -13.3841928  -13.06990708
   -13.69371661]
  [  2.19780488   1.8068525    1.57494774 ...  -3.30029036   3.50399535
    -0.93981417]
  ...
  [ -0.83195122  -1.8529036   -0.73480836 ...  -6.71004646  -5.75576074
    -5.93957027]
  [ -4.5404878   -4.62144019  -6.53334495 ...  -2.44858304  -1.09429733
    -0.25810685]
  [  0.15512195   2.02416957  -1.53773519 ...   1.34702671   1.96131243
     1.9175029 ]]

 [[  5.06401858  -1.32455285 -13.53455285 ... -13.9869338   -3.03312427
    -5.75074332]
  [ -0.53159117   3.2198374    0.3298374  ...  -9.04254355  -9.17873403
   -13.72635308]
  [ -2.88890825  -2.21747967   0.99252033 ...   2.63013937  -0.1760511
     4.13632985]
  ...
  [ -1.58183508  -3.9504065   -3.2404065  ...  -6.29278746  -6.48897793
    -7.57659698]
  [ -3.30012776  -2.00869919  -2.32869919 ...  -5.97108014  -7.94727062
    -8.96488966]
  [ -2.1

In [22]:
scaled_X_nomt_train = norm_layer_nomt(X_nomt_train)
scaled_X_nomt_test = norm_layer_nomt(X_nomt_test)
scaled_X_free = norm_layer_free(X_free_test)
scaled_X_nomt_2 = norm_layer_nomt_2(X_nomt_2)

In [23]:
keras.backend.clear_session()


class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Flatten(),
            layers.Dense(1024, activation='gelu'),
            layers.Dense(512, activation='gelu'),
            layers.Dense(64, activation='gelu'),
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(512, activation='gelu'),
            layers.Dense(1024, activation='gelu'),
            layers.Dense(21 * 141, activation='linear'),
            layers.Reshape((21, 141))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


autoencoder = Autoencoder()

In [24]:
opt = keras.optimizers.Adam(learning_rate=1e-3)
autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())
autoencoder.fit(scaled_X_nomt_train, scaled_X_nomt_train,
                epochs=10,
                batch_size=64,
                shuffle=True,
                validation_data=(scaled_X_nomt_test[:1000], scaled_X_nomt_test[:1000]))

Epoch 1/10
79/79 [==============================] - 3s 29ms/step - loss: 0.5941 - val_loss: 0.5534
Epoch 2/10
79/79 [==============================] - 2s 27ms/step - loss: 0.4914 - val_loss: 0.4680
Epoch 3/10
79/79 [==============================] - 2s 27ms/step - loss: 0.4545 - val_loss: 0.4828
Epoch 4/10
79/79 [==============================] - 2s 27ms/step - loss: 0.4030 - val_loss: 0.4221
Epoch 5/10
79/79 [==============================] - 2s 26ms/step - loss: 0.4210 - val_loss: 0.4616
Epoch 6/10
79/79 [==============================] - 2s 26ms/step - loss: 0.4360 - val_loss: 0.4831
Epoch 7/10
79/79 [==============================] - 2s 26ms/step - loss: 0.3996 - val_loss: 0.4256
Epoch 8/10
79/79 [==============================] - 2s 26ms/step - loss: 0.3789 - val_loss: 0.4395
Epoch 9/10
79/79 [==============================] - 2s 28ms/step - loss: 0.3816 - val_loss: 0.3929
Epoch 10/10
79/79 [==============================] - 2s 28ms/step - loss: 0.3629 - val_loss: 0.4131


In [25]:
test_error,freeform_error=calc_reconstruction_error(autoencoder, scaled_X_nomt_test, scaled_X_free)
test_error_1, test_error_2 = calc_reconstruction_error(autoencoder, scaled_X_nomt_test, scaled_X_nomt_2)


###################
##############
0.388851
0.6554856
###################
##############
0.388851
0.41024154


In [26]:
print(calc_accuracy(test_error, freeform_error, np.mean([np.array(test_error).mean(), np.array(freeform_error).mean()])))
print(calc_accuracy(test_error_1, test_error_2, np.mean([np.array(test_error_1).mean(), np.array(test_error_2).mean()])))


0.7635482030804336
0.5755144750610394


In [27]:
calc_accuracy(test_error, freeform_error, 1)

0.5022818026240731

# Convolutional Autoencoder

In [21]:
X_nomt_train = np.moveaxis(X_nomt_train, 1, 2)
X_nomt_test = np.moveaxis(X_nomt_test, 1, 2)
X_free_test = np.moveaxis(X_free_test, 1, 2)
X_nomt_2 = np.moveaxis(X_nomt_2, 1, 2)
print(X_nomt_train.shape)
print(X_nomt_test.shape)
print(X_free_test.shape)
print(X_nomt_2.shape)

(5000, 141, 21)
(1753, 141, 21)
(1753, 141, 21)
(1114, 141, 21)


In [22]:
norm_layer_nomt = layers.Normalization()
norm_layer_free = layers.Normalization()
norm_layer_nomt_2 = layers.Normalization()

norm_layer_nomt.adapt(X_nomt_train.astype(float))
norm_layer_free.adapt(X_free_test.astype(float))
norm_layer_nomt_2.adapt(X_nomt_2.astype(float))

print(X_nomt_train)
print(np.max(X_nomt_train))
print(np.max(X_free_test))
print(np.max(norm_layer_nomt(X_nomt_train)))
print(np.max(norm_layer_free(X_free_test)))

[[[-5.14212544e+00 -1.00206620e+01  5.80714286e+00 ...  1.99885017e+00
    3.98982578e+00  6.44031359e+00]
  [-8.47926829e+00 -6.29780488e+00 -1.41000000e+00 ...  7.11707317e-01
    4.51268293e+00  8.73317073e+00]
  [ 1.02840650e+01  2.43552846e+00  1.62333333e+00 ...  1.13504065e+00
    2.66601626e+00  4.49650407e+00]
  ...
  [-1.00240302e+01  3.59743322e+00  1.69523810e+00 ...  5.63694541e+00
    3.15792102e+00  3.38408827e-01]
  [ 4.24501742e+00  9.67648084e+00  5.34428571e+00 ...  5.28599303e+00
    1.57696864e+00 -7.42543554e-01]
  [ 1.18354936e+01  4.06695703e+00  2.34761905e-01 ...  4.12646922e+00
   -1.11255517e+00 -7.82067364e-01]]

 [[-1.78118467e+00 -2.76459930e+00 -9.50452962e-01 ... -3.34728223e+00
    8.64668990e-01  5.32418118e+00]
  [-1.64518002e-01 -6.97932636e-01  1.82621370e+00 ... -8.50615563e-01
    1.34133566e+00  4.50847851e-01]
  [-4.26898955e-01 -1.46031359e+00  2.56383275e+00 ... -8.82996516e-01
    2.59895470e+00 -1.15331010e-02]
  ...
  [ 8.04976771e+00  1.1

In [23]:
scaled_X_nomt_train = norm_layer_nomt(X_nomt_train)
scaled_X_nomt_test = norm_layer_nomt(X_nomt_test)
scaled_X_free = norm_layer_free(X_free_test)
scaled_X_nomt_2 = norm_layer_nomt_2(X_nomt_2)

In [116]:
keras.backend.clear_session()
encoding_dim = 50

class ConvAutoencoder(Model):
    def __init__(self, encoding_dim):
        super(ConvAutoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(X_nomt_train.shape[1], X_nomt_train.shape[2])),  # 141, 21
            layers.Conv1D(42, 3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.MaxPooling1D(pool_size=2, padding="same"),
            layers.Conv1D(84, 3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.MaxPooling1D(pool_size=2, padding="same"),
            layers.Flatten(),
            layers.Dense(encoding_dim, activation="relu")
        ])

        self.decoder = tf.keras.Sequential([
            layers.Dense(36*84, activation="relu", use_bias=False),
            layers.Reshape((36, 84)),
            layers.Conv1DTranspose(84, kernel_size=3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.UpSampling1D(),
            layers.Conv1DTranspose(42, kernel_size=3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.UpSampling1D(),
            layers.Conv1D(X_nomt_train.shape[2], kernel_size=3, activation=None, padding='same'),
            layers.Cropping1D(cropping=(2,1))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


conv_autoencoder = ConvAutoencoder(encoding_dim)


In [77]:
opt = keras.optimizers.Adam(learning_rate=1e-3)
conv_autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())

In [78]:
conv_autoencoder.encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 141, 42)           2688      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 141, 42)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 71, 42)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 71, 84)            10668     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 71, 84)            0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 36, 84)           0         
 1D)                                                    

In [79]:
# conv_autoencoder.decoder.summary()

In [80]:
conv_autoencoder.fit(scaled_X_nomt_train, scaled_X_nomt_train,
                     epochs=10,
                     batch_size=64,
                     shuffle=True,
                     validation_data=(scaled_X_nomt_test, scaled_X_nomt_test))

Epoch 1/10
79/79 [==============================] - 3s 28ms/step - loss: 0.7464 - val_loss: 0.4922
Epoch 2/10
79/79 [==============================] - 2s 25ms/step - loss: 0.5063 - val_loss: 0.4585
Epoch 3/10
79/79 [==============================] - 2s 24ms/step - loss: 0.4638 - val_loss: 0.3975
Epoch 4/10
79/79 [==============================] - 2s 25ms/step - loss: 0.3952 - val_loss: 0.3529
Epoch 5/10
79/79 [==============================] - 2s 25ms/step - loss: 0.3627 - val_loss: 0.3324
Epoch 6/10
79/79 [==============================] - 2s 25ms/step - loss: 0.3408 - val_loss: 0.3205
Epoch 7/10
79/79 [==============================] - 2s 25ms/step - loss: 0.3450 - val_loss: 0.3058
Epoch 8/10
79/79 [==============================] - 2s 25ms/step - loss: 0.3221 - val_loss: 0.3010
Epoch 9/10
79/79 [==============================] - 2s 26ms/step - loss: 0.3213 - val_loss: 0.3006
Epoch 10/10
79/79 [==============================] - 2s 26ms/step - loss: 0.3129 - val_loss: 0.3006


In [81]:
test_error, freeform_error = calc_reconstruction_error(conv_autoencoder, scaled_X_nomt_test, scaled_X_free)
# test_error_1, test_error_2 = calc_reconstruction_error(conv_autoencoder, scaled_X_nomt_test, scaled_X_nomt_2)

###################
##############
0.30056337
0.54203016
###################
##############
0.30056337
0.33237106


In [82]:
print(calc_accuracy(test_error, freeform_error, np.mean([np.array(test_error).mean(), np.array(freeform_error).mean()])))
# print(calc_accuracy(test_error_1, test_error_2, np.mean([np.array(test_error_1).mean(), np.array(test_error_2).mean()])))

0.8200228180262408
0.5493547261946286


In [83]:
calc_accuracy(test_error, freeform_error, 1)

0.4982886480319452

# Convolutional Autoencoder - Variants
## Increased encoding space

In [117]:
conv_autoencoder = ConvAutoencoder(encoding_dim=256)

opt = keras.optimizers.Adam(learning_rate=1e-3)
conv_autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())

In [118]:
conv_autoencoder.fit(scaled_X_nomt_train, scaled_X_nomt_train,
                     epochs=10,
                     batch_size=64,
                     shuffle=True,
                     validation_data=(scaled_X_nomt_test, scaled_X_nomt_test))

Epoch 1/10
79/79 [==============================] - 3s 29ms/step - loss: 0.6017 - val_loss: 0.4228
Epoch 2/10
79/79 [==============================] - 2s 28ms/step - loss: 0.4334 - val_loss: 0.3878
Epoch 3/10
79/79 [==============================] - 2s 27ms/step - loss: 0.3989 - val_loss: 0.3760
Epoch 4/10
79/79 [==============================] - 2s 28ms/step - loss: 0.3659 - val_loss: 0.3447
Epoch 5/10
79/79 [==============================] - 2s 28ms/step - loss: 0.3394 - val_loss: 0.2955
Epoch 6/10
79/79 [==============================] - 2s 27ms/step - loss: 0.3065 - val_loss: 0.3252
Epoch 7/10
79/79 [==============================] - 2s 27ms/step - loss: 0.3105 - val_loss: 0.2928
Epoch 8/10
79/79 [==============================] - 2s 28ms/step - loss: 0.3020 - val_loss: 0.2732
Epoch 9/10
79/79 [==============================] - 2s 28ms/step - loss: 0.2864 - val_loss: 0.2671
Epoch 10/10
79/79 [==============================] - 2s 28ms/step - loss: 0.2759 - val_loss: 0.2598


In [119]:
test_error, freeform_error = calc_reconstruction_error(conv_autoencoder, scaled_X_nomt_test, scaled_X_free)

###################
##############
0.25977543
0.4511026


In [120]:
print(calc_accuracy(test_error, freeform_error, np.mean([np.array(test_error).mean(), np.array(freeform_error).mean()])))

0.8034797490017114


## More layers

In [106]:
keras.backend.clear_session()
encoding_dim = 50

class ConvAutoencoder(Model):
    def __init__(self, encoding_dim):
        super(ConvAutoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(X_nomt_train.shape[1], X_nomt_train.shape[2])),  # 141, 21
            layers.Conv1D(42, 3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.MaxPooling1D(pool_size=2, padding="same"),
            layers.Conv1D(84, 3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.MaxPooling1D(pool_size=2, padding="same"),
            layers.Conv1D(168, 3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.MaxPooling1D(pool_size=2, padding="same"),
            layers.Flatten(),
            layers.Dense(encoding_dim, activation="relu")
        ])

        self.decoder = tf.keras.Sequential([
            layers.Dense(18*168, activation="relu", use_bias=False),
            layers.Reshape((18, 168)),
            layers.Conv1DTranspose(168, kernel_size=3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.UpSampling1D(),
            layers.Conv1DTranspose(84, kernel_size=3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.UpSampling1D(),
            layers.Conv1DTranspose(42, kernel_size=3, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.UpSampling1D(),
            layers.Conv1D(X_nomt_train.shape[2], kernel_size=3, activation=None, padding='same'),
            layers.Cropping1D(cropping=(2,1))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


conv_autoencoder = ConvAutoencoder(encoding_dim)

In [107]:
opt = keras.optimizers.Adam(learning_rate=1e-3)
conv_autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())

In [108]:
conv_autoencoder.encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 141, 42)           2688      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 141, 42)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 71, 42)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 71, 84)            10668     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 71, 84)            0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 36, 84)           0         
 1D)                                                    

In [109]:
conv_autoencoder.fit(scaled_X_nomt_train, scaled_X_nomt_train,
                     epochs=10,
                     batch_size=64,
                     shuffle=True,
                     validation_data=(scaled_X_nomt_test, scaled_X_nomt_test))

Epoch 1/10
79/79 [==============================] - 4s 35ms/step - loss: 0.8651 - val_loss: 0.6083
Epoch 2/10
79/79 [==============================] - 2s 31ms/step - loss: 0.6030 - val_loss: 0.5298
Epoch 3/10
79/79 [==============================] - 2s 31ms/step - loss: 0.5463 - val_loss: 0.4833
Epoch 4/10
79/79 [==============================] - 2s 32ms/step - loss: 0.5028 - val_loss: 0.4519
Epoch 5/10
79/79 [==============================] - 2s 32ms/step - loss: 0.4810 - val_loss: 0.4379
Epoch 6/10
79/79 [==============================] - 2s 31ms/step - loss: 0.4567 - val_loss: 0.4166
Epoch 7/10
79/79 [==============================] - 3s 32ms/step - loss: 0.4315 - val_loss: 0.3927
Epoch 8/10
79/79 [==============================] - 2s 31ms/step - loss: 0.4140 - val_loss: 0.3825
Epoch 9/10
79/79 [==============================] - 2s 31ms/step - loss: 0.4031 - val_loss: 0.3732
Epoch 10/10
79/79 [==============================] - 3s 32ms/step - loss: 0.3895 - val_loss: 0.3714


In [110]:
test_error, freeform_error = calc_reconstruction_error(conv_autoencoder, scaled_X_nomt_test, scaled_X_free)

###################
##############
0.37138057
0.6604081


In [111]:
print(calc_accuracy(test_error, freeform_error, np.mean([np.array(test_error).mean(), np.array(freeform_error).mean()])))

0.8054763262977752
